In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns
import re
import random as rd
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from nltk import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import autokeras as ak
from sklearn.model_selection import KFold

import nltk
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def Tokenize(f):     ## Pre-processando a frase

    ## Colocando em minusculo
    ## Retirando a pontuaçao
    ## Retirando as StopWords
    
    f = f.lower().replace('\n', '').replace('-','').replace('#','').replace('.','').replace(',','').replace('!','').replace('r\n','').replace('  ','').replace('https','').replace('rt','').replace('rn','')
    token = RegexpTokenizer(r"\w+")
    f = token.tokenize(f)
    
    stop_words = set(stopwords.words('portuguese'))
    
    new_word = [word for word in f if not word in stop_words]
    
    return ' '.join(new_word)

def remove_user(frase):
    frase = re.sub('@\w+','',frase)
    frase = re.sub('{https}[^ ]+','',frase)
    frase = re.sub('https\w+','',frase)
    # re.sub('#\w+','',frase)
    return frase

def pre_X(frases):
    lista = []
    
    for frase in frases:
        lista.append(frase)
        
    return lista

def pre_Y(number):
    lista = []
    
    for numb in number:
        lista.append(numb)
    
    return lista

def set_array(frases):
    
    vocab = []
    palavras = []
    for frase in frases:
        
        text_array = remove_user(frase)
        text_array = Tokenize(text_array)
        text_array = text_array.split(' ')
        for i in range(len(text_array)):
            vocab.append(text_array[i])
        
    return vocab


## TASH

In [3]:
df1 = pd.read_csv("tash-pt.csv")
df1 = df1.dropna()
df1 = df1.drop(columns=['id_twitter'])


In [4]:
df1['text'] = df1['text'].apply(remove_user)
df1['text'] = df1['text'].apply(Tokenize)

In [5]:
text_TASH = df1['text']
sentiment_ = np.asarray(df1['sentiment'])

count_vect = CountVectorizer()
X_text_TASH = count_vect.fit_transform(text_TASH)

tfidf_transformer = TfidfTransformer()

X_train_TASH_ = tfidf_transformer.fit_transform(X_text_TASH) # Aplicando o TF-IDF
X_train_TASH_.shape

(2787, 11525)

## TWEETSENT

In [6]:
df_tweet= pd.read_csv('export_TweetSentBR.csv')
df_tweet

,id,id_twitter,text,sentiment
0,1343,863044774588272640,Que coisa linda! O Programa #encontro estava m...,1
1,1344,865583716088766467,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,1345,865063232201011201,Mr. CATRA @OficialMrCatra lançando sua nova mú...,1
3,1346,864668391008763905,quem viu aquela lutadora modela barbuda tatuad...,0
4,1347,865572794016378882,Tô passada com esse cara.... quanta merda pode...,-1
...,...,...,...,...
11565,12908,864636619000877056,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11566,12909,863581588713603072,MALUCO! Uma coisa que eu não tenho coragem é e...,-1
11567,12910,864831041349054464,#MaisVoce @ANAMARIABRAGA está linda @RedeGlobo,1
11568,12911,863042798575951872,"Que orgulho de ti, @sportrecife! #Encontro",1


In [7]:
df_remove = df_tweet[df_tweet['sentiment'] == '-']
df_tweet = df_tweet.drop(df_remove.index)

df_tweet['text'] = df_tweet['text'].apply(remove_user)
df_tweet['text'] = df_tweet['text'].apply(Tokenize)

In [8]:
df_tweet['sentiment'] = df_tweet['sentiment'].apply(lambda x: int(x))

In [9]:
text_Tweetsent = df_tweet['text']
polarity_ = np.asarray(df_tweet['sentiment'])

In [10]:
X_text = count_vect.transform(text_Tweetsent)

X_test_TWEET = tfidf_transformer.transform(X_text) # Aplicando o TF-IDF
X_test_TWEET.shape

(11533, 11525)

## KANSOAN

In [11]:
df = pd.read_csv("data.txt", sep=',', header=None, names=['text','sentiment'])

df_remove = df[df['sentiment'] == '#Inveja'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Raiva'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Ironia'].index
df = df.drop(df_remove)


def binario(termo):
    if termo == '#Feliz' or termo == '#Amor':
        return 1
    elif termo == '#Triste' or termo == '#Chateado':
        return 0       
    
df['sentiment'] = df['sentiment'].apply(binario)

In [12]:
df['text'] = df['text'].apply(remove_user)
df['text'] = df['text'].apply(Tokenize)

In [13]:
text_KANSOAN = df['text']
sentiment = np.asarray(df['sentiment'])

X_text_KANSOAN = count_vect.transform(text_KANSOAN)
X_text_KANSOAN_ = tfidf_transformer.transform(X_text_KANSOAN) # Aplicando o TF-IDF
X_text_KANSOAN_.shape

(9224, 11525)

## UNILEX

In [14]:
df1 = pd.read_excel('./rotulação/TweetsPolitical01 OK OK.xlsx')
df2 = pd.read_excel('./rotulação/TweetsPolitical02 OK OK.xlsx')
df3 = pd.read_excel('./rotulação/TweetsPolitical03 OK OK.xlsx')
df4 = pd.read_excel('./rotulação/TweetsPolitical04 OK OK.xlsx')
df5 = pd.read_excel('./rotulação/TweetsPolitical05 OK OK.xlsx')
df6 = pd.read_excel('./rotulação/TweetsPolitical06 OK OK.xlsx')
df7 = pd.read_excel('./rotulação/TweetsPolitical07 OK OK.xlsx')
df8 = pd.read_excel('./rotulação/TweetsPolitical08 OK OK.xlsx')
df9 = pd.read_excel('./rotulação/TweetsPolitical09 OK OK.xlsx')
df10 = pd.read_excel('./rotulação/TweetsPolitical10 OK OK.xlsx')
df11 = pd.read_excel('./rotulação/TweetsPolitical11 OK OK.xlsx')
df12 = pd.read_excel('./rotulação/TweetsPolitical12 OK OK.xlsx')
df13 = pd.read_excel('./rotulação/TweetsPolitical13 OK OK.xlsx')
df14 = pd.read_excel('./rotulação/TweetsPolitical14 OK OK.xlsx')
df15 = pd.read_excel('./rotulação/TweetsPolitical15 OK OK.xlsx')

lista = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15]

df_all = pd.concat(lista)
df_all.shape

(12670, 2)

In [15]:
df_all = df_all.dropna()

In [16]:
df_all.drop(df_all[df_all['Polaridade'] == 11].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == -2].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == 10].index, inplace=True)

df_all['Polaridade'].unique()

array([ 0.,  1., -1.])

In [17]:
df_all.head()

,Tweet,Polaridade
0,- #CaoNossoDeCadaDia #Novo - Vanessa Mandotti ...,0.0
1,"- Bola pra frente que amanhã é outro dia, outr...",1.0
2,- Cara de mal? Acho que não... Apenas um corte...,1.0
3,""" #PCdoB O PARTIDO QUE FAZ A DIFERENÇA: LEAL E...",1.0
4,"""@camaradamae: #PCdoB o Partido da coragem ! ...",1.0


In [18]:
df_all['Tweet'] = df_all['Tweet'].apply(remove_user)
df_all['Tweet'] = df_all['Tweet'].apply(Tokenize)
polarity = np.asarray(df_all['Polaridade'])
Tweet = df_all['Tweet']


In [19]:
Tweet = count_vect.transform(Tweet)
X_test_UNILEX = tfidf_transformer.transform(Tweet) # Aplicando o TF-IDF
X_test_UNILEX.shape

(12626, 11525)

## NB - TRAIN -> TASH- TEST -> TWEET

In [20]:
## TIRAR OS SPLITS , até no treino

In [21]:
clf = MultinomialNB()
clf.fit(X_train_TASH_, sentiment_)
y_pred = clf.predict(X_test_TWEET)

print(classification_report(polarity_,y_pred))

              precision    recall  f1-score   support

          -1       0.41      0.25      0.31      3388
           0       0.29      0.60      0.39      2928
           1       0.63      0.41      0.50      5217

    accuracy                           0.41     11533
   macro avg       0.44      0.42      0.40     11533
weighted avg       0.48      0.41      0.41     11533



## NB - TRAIN -> TASH - TEST -> UNILEX

In [22]:
clf = MultinomialNB()
clf.fit(X_train_TASH_, sentiment_)
y_pred = clf.predict(X_test_UNILEX)

print(classification_report(polarity,y_pred))

              precision    recall  f1-score   support

        -1.0       0.45      0.55      0.50      4186
         0.0       0.44      0.56      0.49      4740
         1.0       0.43      0.17      0.24      3700

    accuracy                           0.44     12626
   macro avg       0.44      0.43      0.41     12626
weighted avg       0.44      0.44      0.42     12626



## NB - TRAIN -> TASH - TEST -> KANSOAN

In [23]:
def func(num):
    if num == 0:
        return -1
    else:
        return 1

sentiment = df['sentiment'].apply(func)


In [24]:
clf = MultinomialNB()
clf.fit(X_train_TASH_, sentiment_)
y_pred = clf.predict(X_text_KANSOAN_)

print(classification_report(sentiment,y_pred,zero_division=True))

              precision    recall  f1-score   support

          -1       0.90      0.62      0.74      4504
           0       0.00      1.00      0.00         0
           1       0.86      0.74      0.80      4720

    accuracy                           0.68      9224
   macro avg       0.59      0.79      0.51      9224
weighted avg       0.88      0.68      0.77      9224



## SVM - TRAIN->TASH - TEST-> TWEET

In [25]:
clf = SVC().fit(X_train_TASH_, sentiment_) 
y_pred = clf.predict(X_test_TWEET)

print(classification_report(polarity_,y_pred))

              precision    recall  f1-score   support

          -1       0.45      0.21      0.28      3388
           0       0.28      0.76      0.42      2928
           1       0.70      0.29      0.41      5217

    accuracy                           0.38     11533
   macro avg       0.48      0.42      0.37     11533
weighted avg       0.52      0.38      0.37     11533



## SVM T TASH T UNILEX

In [26]:
clf = SVC().fit(X_train_TASH_, sentiment_) 
y_pred = clf.predict(X_test_UNILEX)

print(classification_report(polarity,y_pred))

              precision    recall  f1-score   support

        -1.0       0.50      0.36      0.42      4186
         0.0       0.43      0.79      0.56      4740
         1.0       0.54      0.11      0.19      3700

    accuracy                           0.45     12626
   macro avg       0.49      0.42      0.39     12626
weighted avg       0.48      0.45      0.40     12626



## SVM T UNILEX T KANSOAN

In [27]:
clf = SVC().fit(X_train_TASH_, sentiment_) 
y_pred = clf.predict(X_text_KANSOAN_)

print(classification_report(sentiment,y_pred, zero_division=True))

              precision    recall  f1-score   support

          -1       0.92      0.58      0.71      4504
           0       0.00      1.00      0.00         0
           1       0.92      0.71      0.80      4720

    accuracy                           0.64      9224
   macro avg       0.61      0.76      0.50      9224
weighted avg       0.92      0.64      0.75      9224



## RL T TASH T TWEET

In [28]:
clf = LogisticRegression(max_iter=1000).fit(X_train_TASH_, sentiment_) 
y_pred = clf.predict(X_test_TWEET)

print(classification_report(polarity_,y_pred))

              precision    recall  f1-score   support

          -1       0.42      0.27      0.33      3388
           0       0.30      0.60      0.40      2928
           1       0.63      0.41      0.50      5217

    accuracy                           0.42     11533
   macro avg       0.45      0.43      0.41     11533
weighted avg       0.49      0.42      0.42     11533



## RL T TASH T UNILEX

In [29]:
clf = LogisticRegression(max_iter=1000).fit(X_train_TASH_, sentiment_) 
y_pred = clf.predict(X_test_UNILEX)

print(classification_report(polarity,y_pred))

              precision    recall  f1-score   support

        -1.0       0.45      0.54      0.49      4186
         0.0       0.45      0.58      0.50      4740
         1.0       0.46      0.19      0.26      3700

    accuracy                           0.45     12626
   macro avg       0.45      0.44      0.42     12626
weighted avg       0.45      0.45      0.43     12626



## RL T UNILEX T KANSOAN

In [30]:
clf = LogisticRegression(max_iter=1000).fit(X_train_TASH_, sentiment_) 
y_pred = clf.predict(X_text_KANSOAN_)

print(classification_report(sentiment,y_pred, zero_division=True))

              precision    recall  f1-score   support

          -1       0.91      0.66      0.77      4504
           0       0.00      1.00      0.00         0
           1       0.89      0.79      0.83      4720

    accuracy                           0.73      9224
   macro avg       0.60      0.82      0.53      9224
weighted avg       0.90      0.73      0.80      9224



In [31]:
one = OneHotEncoder(sparse=False)
teste_ = one.fit_transform(np.asarray(sentiment).reshape(-1,1))
nome = pd.DataFrame(teste_)
nome['coluna'] = 0
nome.rename(columns={1: 2, 'coluna':1} , inplace=True)
Y_KANSOAN = nome[[0,1,2]].to_numpy()

## TOTALMENT.C T TASH T TWEET

In [32]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(10, activation='tanh'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(3 , activation='softmax')
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(sentiment_).reshape(-1,1))
y_one_test = one.fit_transform(polarity_.reshape(-1,1))

fit = model.fit(X_train_TASH_.todense(), y_one, epochs=5, validation_data=(X_test_TWEET.todense(), y_one_test))

Epoch 1/5
88/88 [==============================] - 2s 18ms/step - loss: 1.0965 - accuracy: 0.3671 - val_loss: 1.1088 - val_accuracy: 0.2539
Epoch 2/5
88/88 [==============================] - 1s 12ms/step - loss: 1.0589 - accuracy: 0.4503 - val_loss: 1.0873 - val_accuracy: 0.3937
Epoch 3/5
88/88 [==============================] - 1s 10ms/step - loss: 0.8229 - accuracy: 0.6903 - val_loss: 1.1290 - val_accuracy: 0.4128
Epoch 4/5
88/88 [==============================] - 1s 14ms/step - loss: 0.4407 - accuracy: 0.8737 - val_loss: 1.3581 - val_accuracy: 0.4019
Epoch 5/5
88/88 [==============================] - 1s 12ms/step - loss: 0.2042 - accuracy: 0.9534 - val_loss: 1.6717 - val_accuracy: 0.4025


In [33]:
predicted = model.predict(X_test_TWEET.todense())

print(classification_report(np.argmax(y_one_test, axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.36      0.36      0.36      3388
           1       0.29      0.45      0.35      2928
           2       0.60      0.40      0.48      5217

    accuracy                           0.40     11533
   macro avg       0.41      0.40      0.40     11533
weighted avg       0.45      0.40      0.41     11533



## UNILEX

In [34]:
predicted = model.predict(X_test_UNILEX.todense())

print(classification_report(np.argmax(one.fit_transform(polarity.reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.40      0.70      0.51      4186
           1       0.44      0.34      0.38      4740
           2       0.39      0.17      0.24      3700

    accuracy                           0.41     12626
   macro avg       0.41      0.40      0.37     12626
weighted avg       0.41      0.41      0.38     12626



## KANSOAN

In [35]:
predicted = model.predict(X_text_KANSOAN_.todense())

print(classification_report(np.argmax(Y_KANSOAN, axis=1), np.argmax(predicted, axis=1), zero_division=True))

              precision    recall  f1-score   support

           0       0.82      0.75      0.78      4504
           1       0.00      1.00      0.00         0
           2       0.85      0.67      0.75      4720

    accuracy                           0.71      9224
   macro avg       0.56      0.81      0.51      9224
weighted avg       0.84      0.71      0.77      9224



## LSTM T TASH T UNILEX

In [36]:
vectorize_layer = TextVectorization(
 max_tokens=15000,
 output_mode='int',
 output_sequence_length=len(max(text_TASH )))

vocab = set_array(text_TASH )

vectorize_layer.adapt(np.unique(vocab))
len(vectorize_layer.get_vocabulary())


11562

In [37]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,mask_zero=True),
    
    tf.keras.layers.LSTM(50, activation='relu' ,return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(25 , activation='tanh', return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(10 , activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(3, activation='softmax')
])


model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(sentiment_).reshape(-1,1))
y_one_test = one.fit_transform(np.asarray(polarity).reshape(-1,1))

fit = model.fit(np.asarray(pre_X(text_TASH)), y_one, epochs=5, batch_size=128 ,validation_data=(np.asarray(pre_X(df_all['Tweet'])), y_one_test))

Epoch 1/5
22/22 [==============================] - 5s 241ms/step - loss: 1.0971 - accuracy: 0.3624 - val_loss: 1.0979 - val_accuracy: 0.3754
Epoch 2/5
22/22 [==============================] - 4s 163ms/step - loss: 1.0908 - accuracy: 0.3653 - val_loss: 1.0992 - val_accuracy: 0.3754
Epoch 3/5
22/22 [==============================] - 4s 163ms/step - loss: 1.0165 - accuracy: 0.4621 - val_loss: 1.1151 - val_accuracy: 0.3244
Epoch 4/5
22/22 [==============================] - 4s 163ms/step - loss: 0.7918 - accuracy: 0.6103 - val_loss: 1.3215 - val_accuracy: 0.3414
Epoch 5/5
22/22 [==============================] - 4s 164ms/step - loss: 0.6341 - accuracy: 0.7180 - val_loss: 1.3603 - val_accuracy: 0.3627


In [38]:
predicted = model.predict(np.asarray(pre_X(df_all['Tweet'])))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.35      0.46      0.40      4186
           1       0.36      0.34      0.35      4740
           2       0.40      0.27      0.33      3700

    accuracy                           0.36     12626
   macro avg       0.37      0.36      0.36     12626
weighted avg       0.37      0.36      0.36     12626



## TWEET

In [39]:
predicted = model.predict(np.asarray(pre_X(text_Tweetsent)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.29      0.39      0.33      3388
           1       0.26      0.38      0.31      2928
           2       0.56      0.31      0.39      5217

    accuracy                           0.35     11533
   macro avg       0.37      0.36      0.35     11533
weighted avg       0.40      0.35      0.35     11533



## KANSOAN

In [40]:
predicted = model.predict(np.asarray(pre_X(text_KANSOAN)))

print(classification_report(np.argmax(Y_KANSOAN, axis=1), np.argmax(predicted, axis=1),zero_division=True))

              precision    recall  f1-score   support

           0       0.66      0.48      0.55      4504
           1       0.00      1.00      0.00         0
           2       0.73      0.51      0.60      4720

    accuracy                           0.50      9224
   macro avg       0.46      0.66      0.39      9224
weighted avg       0.69      0.50      0.58      9224



## CONV1D T TASH T KANSOAN

In [41]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    
    tf.keras.layers.Conv1D(32,6, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')   
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(sentiment_).reshape(-1,1))
y_one_test = Y_KANSOAN

fit = model.fit(np.asarray(pre_X(text_TASH)), y_one, epochs=5, batch_size=128, validation_data=(np.asarray(pre_X(text_KANSOAN)),Y_KANSOAN))

Epoch 1/5
22/22 [==============================] - 1s 32ms/step - loss: 1.0986 - accuracy: 0.3452 - val_loss: 1.1410 - val_accuracy: 0.0000e+00
Epoch 2/5
22/22 [==============================] - 1s 27ms/step - loss: 1.0806 - accuracy: 0.3692 - val_loss: 1.1381 - val_accuracy: 0.0000e+00
Epoch 3/5
22/22 [==============================] - 1s 26ms/step - loss: 1.0290 - accuracy: 0.5468 - val_loss: 1.1322 - val_accuracy: 0.0417
Epoch 4/5
22/22 [==============================] - 1s 25ms/step - loss: 0.8530 - accuracy: 0.8396 - val_loss: 0.9716 - val_accuracy: 0.6983
Epoch 5/5
22/22 [==============================] - 1s 27ms/step - loss: 0.5080 - accuracy: 0.9297 - val_loss: 0.9303 - val_accuracy: 0.6345


In [42]:
predicted = model.predict(np.asarray(pre_X(text_KANSOAN)))

print(classification_report(np.argmax(Y_KANSOAN, axis=1), np.argmax(predicted, axis=1),zero_division=True))

              precision    recall  f1-score   support

           0       0.87      0.49      0.62      4504
           1       0.00      1.00      0.00         0
           2       0.73      0.78      0.75      4720

    accuracy                           0.63      9224
   macro avg       0.53      0.75      0.46      9224
weighted avg       0.80      0.63      0.69      9224



## TWEET

In [43]:
predicted = model.predict(np.asarray(pre_X(text_Tweetsent)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.43      0.30      0.35      3388
           1       0.27      0.32      0.29      2928
           2       0.50      0.53      0.52      5217

    accuracy                           0.41     11533
   macro avg       0.40      0.39      0.39     11533
weighted avg       0.42      0.41      0.41     11533



## UNILEX

In [44]:
predicted = model.predict(np.asarray(pre_X(df_all['Tweet'])))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.43      0.34      0.38      4186
           1       0.44      0.28      0.34      4740
           2       0.31      0.54      0.40      3700

    accuracy                           0.38     12626
   macro avg       0.40      0.39      0.37     12626
weighted avg       0.40      0.38      0.37     12626



## BDR T TASH T TWEET

In [46]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
    
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(sentiment_).reshape(-1,1))
y_one_test = one.fit_transform(np.asarray(polarity_).reshape(-1,1))

fit = model.fit(np.asarray(pre_X(text_TASH)), y_one, epochs=5, batch_size=128 ,validation_data=(np.asarray(pre_X(text_Tweetsent)), y_one_test))

Epoch 1/5
22/22 [==============================] - 5s 220ms/step - loss: 1.0970 - accuracy: 0.3645 - val_loss: 1.1106 - val_accuracy: 0.2539
Epoch 2/5
22/22 [==============================] - 3s 142ms/step - loss: 1.0535 - accuracy: 0.4141 - val_loss: 1.1025 - val_accuracy: 0.3548
Epoch 3/5
22/22 [==============================] - 3s 141ms/step - loss: 0.7762 - accuracy: 0.6247 - val_loss: 1.2033 - val_accuracy: 0.3715
Epoch 4/5
22/22 [==============================] - 3s 142ms/step - loss: 0.4848 - accuracy: 0.8245 - val_loss: 1.3036 - val_accuracy: 0.3826
Epoch 5/5
22/22 [==============================] - 3s 142ms/step - loss: 0.2328 - accuracy: 0.9526 - val_loss: 1.5526 - val_accuracy: 0.3955


In [47]:
predicted = model.predict(np.asarray(pre_X(text_Tweetsent)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1),zero_division=True))

              precision    recall  f1-score   support

           0       0.39      0.27      0.32      3388
           1       0.28      0.57      0.37      2928
           2       0.61      0.38      0.47      5217

    accuracy                           0.40     11533
   macro avg       0.43      0.41      0.39     11533
weighted avg       0.46      0.40      0.40     11533



## UNILEX

In [48]:
predicted = model.predict(np.asarray(pre_X(df_all['Tweet'])))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.42      0.56      0.48      4186
           1       0.44      0.50      0.47      4740
           2       0.41      0.17      0.24      3700

    accuracy                           0.43     12626
   macro avg       0.42      0.41      0.40     12626
weighted avg       0.42      0.43      0.41     12626



## KANSOAN

In [49]:
predicted = model.predict(np.asarray(pre_X(text_KANSOAN)))

print(classification_report(np.argmax(Y_KANSOAN, axis=1), np.argmax(predicted, axis=1),zero_division=True))

              precision    recall  f1-score   support

           0       0.85      0.53      0.65      4504
           1       0.00      1.00      0.00         0
           2       0.81      0.62      0.70      4720

    accuracy                           0.58      9224
   macro avg       0.56      0.72      0.45      9224
weighted avg       0.83      0.58      0.68      9224

